In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix,accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV
from sklearn.feature_selection import SelectKBest, f_classif,chi2
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

df = pd.read_csv('./train1.csv')
X_train = df.drop(['target','id'], axis=1)


In [2]:
y_train = df['target']

In [3]:
X_train,X_test,y_train,y_test = train_test_split(X_train, y_train, test_size=0.3, random_state=29173, stratify=y_train)

In [4]:
X_test.shape

(88863, 65)

In [ ]:
"""df_test = pd.read_csv('test.csv')
X_test = df_test.drop('id',axis=1) 
test_ids = df_test['id']"""

"df_test = pd.read_csv('test.csv')\nX_test = df_test.drop('id',axis=1) "

In [5]:
categorical_cols = [col for col in X_train.columns if col.endswith('_cat')]
X_train[categorical_cols] = X_train[categorical_cols].astype('category')
X_test[categorical_cols] = X_test[categorical_cols].astype('category')
binary_cols = [col for col in X_train.columns if col.endswith('_bin')]
X_train[binary_cols] = X_train[binary_cols].astype(bool)
X_test[binary_cols] = X_test[binary_cols].astype(bool)

In [6]:
print(len(categorical_cols))

14


In [7]:
def simple_imputer(X_train, X_val, categorical_columns=None, numerical_columns=None):
    """
    Impute missing values in train and validation/test sets using only training data statistics.
    - categorical_columns: list of categorical columns to impute with mode
    - numerical_columns: list of numerical columns to impute with mean
    """
    X_train_imputed = X_train.copy()
    X_val_imputed = X_val.copy()

    # Categorical columns
    if categorical_columns:
        for col in categorical_columns:
            if col in X_train_imputed.columns:
                mode_value = X_train_imputed[col].mode(dropna=True)
                if not mode_value.empty:
                    mode_value = mode_value[0]
                    X_train_imputed[col].fillna(mode_value, inplace=True)
                    if col in X_val_imputed.columns:
                        X_val_imputed[col].fillna(mode_value, inplace=True)

    # Numerical columns
    if numerical_columns:
        for col in numerical_columns:
            if col in X_train_imputed.columns:
                mean_value = X_train_imputed[col].mean(skipna=True)
                if pd.notna(mean_value):  # ensure mean is valid
                    X_train_imputed[col].fillna(mean_value, inplace=True)
                    if col in X_val_imputed.columns:
                        X_val_imputed[col].fillna(mean_value, inplace=True)

    return X_train_imputed, X_val_imputed


In [8]:
num_cols = [col for col in X_train.columns if not col.endswith(('_cat', '_bin'))]

In [9]:
X_train_imputed,X_test_imputed = simple_imputer(X_train, X_test, categorical_cols,num_cols)

In [10]:
X_train_imputed['ps_car_12'].isnull().sum()

np.int64(0)

In [59]:
X_test_imputed.isna().sum()[X_test.isna().sum() > 0]


ps_ind_02_cat    0
ps_ind_04_cat    0
ps_ind_05_cat    0
ps_car_01_cat    0
ps_car_03_cat    0
ps_car_05_cat    0
ps_car_07_cat    0
ps_car_09_cat    0
ps_reg_03        0
ps_car_11        0
ps_car_12        0
ps_car_14        0
feature4         0
dtype: int64

In [ ]:
baseline_model = AdaBoostClassifier(
    estimator=DecisionTreeClassifier(max_depth=1),  
    n_estimators=100,
    learning_rate=.1,
    random_state=29173
)

baseline_model.fit(X_train_imputed, y_train)
y_pred_proba = baseline_model.predict_proba(X_test_imputed)[:, 1]

roc_auc = roc_auc_score(y_test, y_pred_proba)

print("Baseline AUROC:", roc_auc)

Baseline AUROC: 0.6144934048001933


In [ ]:
param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.05, 0.1, 0.5, 1.0],
    'estimator__max_depth': [1],
    'estimator__min_samples_split': [2, 10],
    'estimator__min_samples_leaf': [1, 2]
}


In [ ]:

base_estimator = DecisionTreeClassifier(random_state=29173)

ada = AdaBoostClassifier(estimator=base_estimator)

grid_search = GridSearchCV(
    estimator=ada,
    param_grid=param_grid,
    cv=3,
    scoring='roc_auc',
    n_jobs=-1,
    verbose=2
)

grid_search.fit(X_train_imputed, y_train)

print("Best parameters:", grid_search.best_params_)
print("Best roc_auc score:", grid_search.best_score_)


Fitting 3 folds for each of 180 candidates, totalling 540 fits


In [12]:
# X_num: only numerical columns
X_num = X_train_imputed[num_cols]

# Apply ANOVA F-test
selector = SelectKBest(score_func=f_classif, k='all')  # 'all' gives scores for all features
selector.fit(X_num, y_train)

# Get scores and p-values
anova_scores = pd.DataFrame({
    'Feature': X_num.columns,
    'F_score': selector.scores_,
    'p_value': selector.pvalues_
}).sort_values(by='F_score', ascending=False)

significant_num_features = anova_scores[anova_scores['p_value'] <= 0.01]['Feature'].tolist()

print(f"Significant num features (p <= 0.01):")
print(significant_num_features)

Significant num features (p <= 0.01):
['ps_car_13', 'ps_reg_02', 'ps_car_12', 'feature4', 'ps_reg_03', 'feature2', 'ps_car_15', 'ps_ind_15', 'ps_reg_01', 'ps_ind_01', 'feature5', 'ps_car_14', 'feature7', 'ps_ind_03', 'ps_calc_01']


d:\ML project\.venv\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:110: UserWarning: Features [26] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
d:\ML project\.venv\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:111: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


In [14]:
# X_cat: select only categorical
X_cat = X_train_imputed[categorical_cols].copy()


# Apply Chi-Square test
chi_selector = SelectKBest(score_func=chi2, k='all')
chi_selector.fit(X_cat, y_train)

# Create a DataFrame with scores and p-values
chi_scores_cat = pd.DataFrame({
    'Feature': X_cat.columns,
    'Chi2_score': chi_selector.scores_,
    'p_value': chi_selector.pvalues_
}).sort_values(by='Chi2_score', ascending=False)

significant_cat_features = chi_scores_cat[chi_scores_cat['p_value'] <= 0.01]['Feature'].tolist()

print(f"Significant cat features (p <= 0.01):")
print(significant_cat_features)


Significant cat features (p <= 0.01):
['ps_car_04_cat', 'ps_ind_05_cat', 'ps_car_11_cat', 'ps_car_06_cat', 'ps_car_01_cat', 'ps_car_02_cat', 'ps_ind_04_cat', 'ps_car_08_cat', 'ps_car_09_cat', 'ps_car_05_cat']


In [15]:
# X_bin: select only binary columns
X_bin = X_train[binary_cols].copy()


# Apply Chi-Square test
chi_selector = SelectKBest(score_func=chi2, k='all')
chi_selector.fit(X_bin, y_train)

# Create a DataFrame with scores and p-values
chi_scores_bin = pd.DataFrame({
    'Feature': X_bin.columns,
    'Chi2_score': chi_selector.scores_,
    'p_value': chi_selector.pvalues_
}).sort_values(by='Chi2_score', ascending=False)

significant_bin_features = chi_scores_bin[chi_scores_bin['p_value'] <= 0.01]['Feature'].tolist()

print(f"Significant binary features (p <= 0.01):")
print(significant_bin_features)


Significant binary features (p <= 0.01):
['ps_ind_17_bin', 'ps_ind_07_bin', 'ps_ind_06_bin', 'ps_ind_16_bin', 'ps_ind_08_bin', 'ps_ind_09_bin', 'ps_ind_12_bin']


In [22]:
# Combine numerical, categorical, and binary top features
selected_features = significant_num_features + significant_cat_features + significant_bin_features
print(selected_features)


['ps_car_13', 'ps_reg_02', 'ps_car_12', 'feature4', 'ps_reg_03', 'feature2', 'ps_car_15', 'ps_ind_15', 'ps_reg_01', 'ps_ind_01', 'feature5', 'ps_car_14', 'feature7', 'ps_ind_03', 'ps_calc_01', 'ps_car_04_cat', 'ps_ind_05_cat', 'ps_car_11_cat', 'ps_car_06_cat', 'ps_car_01_cat', 'ps_car_02_cat', 'ps_ind_04_cat', 'ps_car_08_cat', 'ps_car_09_cat', 'ps_car_05_cat', 'ps_ind_17_bin', 'ps_ind_07_bin', 'ps_ind_06_bin', 'ps_ind_16_bin', 'ps_ind_08_bin', 'ps_ind_09_bin', 'ps_ind_12_bin']


In [ ]:
baseline_model.fit(X_train_imputed[selected_features], y_train)
y_pred_proba_chi_anova = baseline_model.predict_proba(X_test_imputed[selected_features])[:, 1]
val_auroc = roc_auc_score(y_test, y_pred_proba_chi_anova)
print(f"Validation AUROC: {val_auroc:.4f}")
"""
submission = pd.DataFrame({'id': test_ids, 'target': y_pred_proba_chi_anova})
submission.to_csv('submission_chi_anova.csv', index=False) """

Validation AUROC: 0.6145


"\nsubmission = pd.DataFrame({'id': test_ids, 'target': y_pred_proba_chi_anova})\nsubmission.to_csv('submission_chi_anova.csv', index=False) "

In [18]:
param_grid = {
    'n_estimators': [100],
    'learning_rate': [0.05, 0.1, 1.0],
    'estimator__max_depth': [1, 2],
    'estimator__min_samples_split': [2, 10],
    'estimator__min_samples_leaf': [1,2]
}

base_estimator = DecisionTreeClassifier(random_state=29173)

ada = AdaBoostClassifier(
    estimator=base_estimator,
    random_state=29173
)

grid_search = GridSearchCV(
    estimator=ada,
    param_grid=param_grid,
    cv=3,
    scoring='roc_auc',
    n_jobs=-1,
    verbose=2
)

grid_search.fit(X_train_imputed[selected_features], y_train)
print("Best Parameters:", grid_search.best_params_)
print("Best AUROC (CV):", grid_search.best_score_)




Fitting 3 folds for each of 24 candidates, totalling 72 fits
Best Parameters: {'estimator__max_depth': 1, 'estimator__min_samples_leaf': 1, 'estimator__min_samples_split': 2, 'learning_rate': 1.0, 'n_estimators': 100}
Best AUROC (CV): 0.622208034765834


In [21]:
tuned_adaboost_model_chi_anova_tuned = AdaBoostClassifier(
    estimator=DecisionTreeClassifier(max_depth=1, min_samples_leaf=1, min_samples_split=2),
    learning_rate=1,
    n_estimators=100
)
tuned_adaboost_model_chi_anova_tuned.fit(X_train_imputed[selected_features], y_train)
y_pred_proba_chi_anova = tuned_adaboost_model_chi_anova_tuned.predict_proba(X_test_imputed[selected_features])[:, 1]
val_auroc = roc_auc_score(y_test, y_pred_proba_chi_anova)
print(f"Validation AUROC: {val_auroc:.4f}")
"""
submission = pd.DataFrame({'id': test_ids, 'target': y_pred_proba_chi_anova})
submission.to_csv('submission_chi_anova.csv', index=False) """

Validation AUROC: 0.6262


"\nsubmission = pd.DataFrame({'id': test_ids, 'target': y_pred_proba_chi_anova})\nsubmission.to_csv('submission_chi_anova.csv', index=False) "

In [ ]:
tuned_adaboost_model = AdaBoostClassifier(
    estimator=DecisionTreeClassifier(max_depth=2, min_samples_leaf=1, min_samples_split=2),
    learning_rate=0.5,
    n_estimators=200
)
tuned_adaboost_model.fit(X_train_imputed[selected_features], y_train)
y_pred_proba_chi_anova = tuned_adaboost_model.predict_proba(X_test_imputed[selected_features])[:, 1]
val_auroc = roc_auc_score(y_test, y_pred_proba_chi_anova)
print(f"Validation AUROC: {val_auroc:.4f}")
"""
submission = pd.DataFrame({'id': test_ids, 'target': y_pred_proba_chi_anova})
submission.to_csv('submission_chi_anova.csv', index=False) """

Validation AUROC: 0.6278


"\nsubmission = pd.DataFrame({'id': test_ids, 'target': y_pred_proba_chi_anova})\nsubmission.to_csv('submission_chi_anova.csv', index=False) "

In [ ]:
# Evaluate on test
best_dt = grid_search.best_estimator_
y_pred_proba_best_chi_anova = best_dt.predict_proba(X_test[selected_features])[:, 1]
submission = pd.DataFrame({'id': test_ids, 'target': y_pred_proba_best_chi_anova})
submission.to_csv('submission_best_chi_anova.csv', index=False)

In [ ]:
#wrapper Backward
dt = DecisionTreeClassifier(
    class_weight=None, 
    criterion='gini', 
    max_depth=5, 
    min_samples_leaf=5, 
    min_samples_split=2,
    random_state=29173
)
dt.fit(X_train[['ps_ind_05_cat', 'ps_car_04_cat', 'ps_ind_17_bin', 'ps_reg_03', 'ps_car_14', 'ps_car_15']], y_train)

y_pred_proba_backward_wrapper = dt.predict_proba(X_test[['ps_ind_05_cat', 'ps_car_04_cat', 'ps_ind_17_bin', 'ps_reg_03', 'ps_car_14', 'ps_car_15']])[:, 1]
submission = pd.DataFrame({'id': test_ids, 'target': y_pred_proba_backward_wrapper})
submission.to_csv('submission_backward_wrapper.csv', index=False)



In [ ]:
forward_cols = ['ps_car_13', 'ps_ind_17_bin', 'feature4', 'ps_ind_15', 'ps_car_04_cat','ps_reg_02', 'ps_calc_20_bin']
dt.fit(X_train[forward_cols], y_train)

y_pred_proba_forward_wrapper = dt.predict_proba(X_test[forward_cols])[:, 1]
submission = pd.DataFrame({'id': test_ids, 'target': y_pred_proba_forward_wrapper})
submission.to_csv('submission_forward_wrapper.csv', index=False)


In [ ]:
X_train_encoded = pd.get_dummies(X_train, columns=categorical_cols, drop_first=False)
print("Encoded DataFrame shape:", X_train_encoded.shape)

In [ ]:
X_test_encoded = pd.get_dummies(X_test, columns=categorical_cols, drop_first=False)
print("Encoded DataFrame shape:", X_test_encoded.shape)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_encoded)
X_test_scaled = scaler.transform(X_test_encoded)
pca = PCA(n_components=.8)

X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)



X_train_pca_df = pd.DataFrame(X_train_pca, columns=[f'PC{i+1}' for i in range(pca.n_components_)])
X_test_pca_df = pd.DataFrame(X_test_pca, columns=[f'PC{i+1}' for i in range(pca.n_components_)])


In [ ]:

dtPCA = DecisionTreeClassifier(class_weight= None, criterion= 'gini', max_depth= 5, min_samples_leaf= 5, min_samples_split= 2)
dtPCA.fit(X_train_pca_df, y_train)
y_pred_proba_pca = dtPCA.predict_proba(X_test_pca_df)[:, 1]

# saving the results
submission = pd.DataFrame({'id': test_ids, 'target': y_pred_proba_pca})
submission.to_csv('submission_pca.csv', index=False)


In [ ]:
""" param_grid = { 
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 5, 10, 15, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5],
    'class_weight': [None, 'balanced']
}
grid_search = GridSearchCV(
    estimator=dt,
    param_grid=param_grid,
    scoring='roc_auc',
    cv=5,               
    n_jobs=-1,
    verbose=2
)

grid_search.fit(X_train_pca_df, y_train)
print("Best Parameters:", grid_search.best_params_)
print("Best AUROC (CV):", grid_search.best_score_)  """



In [ ]:
""" param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 5, 10, 15, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5],
    'class_weight': [None, 'balanced']
}
grid_search = RandomizedSearchCV(
    estimator=dt,
    param_distributions=param_grid,
    scoring='roc_auc',
    cv=5,
    n_jobs=-1,
    verbose=2,
    n_iter=20, 
    random_state=29173
)

grid_search.fit(X_train_pca_df, y_train)
print("Best Parameters:", grid_search.best_params_)
print("Best AUROC (CV):", grid_search.best_score_)
"""